<a href="https://colab.research.google.com/github/Angeal13/ClassifierML/blob/master/IrisFlowerClassificator_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

%tensorflow_version 2.x

In [2]:
from __future__ import absolute_import,division,print_function,unicode_literals

import tensorflow as tf

import pandas as pd

In [3]:
CSV_COLUMN_NAMES=['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
SPECIES=['Setosa','Versicolor','Virginia'] #Defince constant that will help later

In [4]:
train_path=tf.keras.utils.get_file(
    "iris_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path=tf.keras.utils.get_file(
    "iris_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
test  = pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [5]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
train_y = train.pop('Species')
test_y  = test.pop('Species')

In [7]:
def input_fn(features,labels,training=True,batch_size=256):
  dataset=tf.data.Dataset.from_tensor_slices((dict(features),labels)) #Covert the inputs into a DataSest

  if training: #Shuffle and repeat if you are in training mode
    dataset=dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)

In [8]:
#Feature columns describe how to use the input
my_feature_columns=[]
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [9]:
#Build a DNN with n-hidden layers with a certain number of nodes to choose with is a certain number of classes
classifier=tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30,10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmph75i7zfy', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [10]:
#Training the model unsig LAMBDA fuction
#We use LAMBDA becuse we do not have a function that would retunr the input_fn object to us
#Usig Steps instead of Epoch, to only look at the number of objects desire, both can be used for proceding adn processing
classifier.train(
    input_fn=lambda:input_fn(train,train_y,training=True), steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmph75i7zfy/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 4.701971, step = 0
INFO:tensorflow:global_step/sec: 426.768
INFO:tensorflow:loss = 3.0692477, step = 100 (0.236 sec)
INFO:tensorflow:global_step/sec: 557.189
INFO:tensorflow:loss = 2.302468, step = 200 (0.184 sec)
INFO:tensorflow:global_step/sec: 

In [11]:
#Evaluate the model Using LAMBDA as explain in the previous set
eval_result=classifier.evaluate(
    input_fn=lambda:input_fn(test,test_y,training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-08-14T19:41:48
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmph75i7zfy/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.23086s
INFO:tensorflow:Finished evaluation at 2021-08-14-19:41:48
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.6666667, average_loss = 0.6654516, global_step = 5000, loss = 0.6654516
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmph75i7zfy/model.ckpt-5000

Test set accuracy: 0.667



In [12]:
def input_fn(features,batch_size=256):
  #Convert the inputs to a DataSet without Labels.
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features=['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict={}

print("Please type numeric values as prompted")
for feature in features:
  valid=True
  while valid:
    val=input(feature + ": ")
    if not val.isdigit():valid =False
  
  predict[feature]=[float(val)]

predictions=classifier.predict(input_fn=lambda:input_fn(predict))
for pred_dict in predictions:
  class_id=pred_dict['class_ids'][0]
  probability= pred_dict['probabilities'][class_id]

  print('Predictions is "{}" ({:.1f}%)'.format(
      SPECIES[class_id],100 * probability))

Please type numeric values as prompted
SepalLength: 0.2
SepalWidth: 0.6
PetalLength: 1.2
PetalWidth: 0.7
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmph75i7zfy/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predictions is "Virginia" (44.4%)
